# Simple LLM Chain

## Put Your Together AI key here

In [ ]:
import os
os.environ["TOGETHER_API_KEY"] = "paste_your_api_key_here"

## Install Packages

In [ ]:
!pip3 install langchain langchain_openai

## Set up the LLM

In [ ]:
import os
from langchain_openai import ChatOpenAI

#model_name = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-SFT"
#model_name = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
#model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
#model_name="mistralai/Mistral-7B-Instruct-v0.1"
#model_name="mistralai/Mistral-7B-Instruct-v0.2"
model_name = "openchat/openchat-3.5-1210"
#model_name = "Qwen/Qwen1.5-72B-Chat"
#model_name = "cognitivecomputations/dolphin-2.5-mixtral-8x7b"
#model_name="databricks/dbrx-instruct"
#model_name="mistralai/Mixtral-8x22B"

print("Provider: TogetherAI")
print("Model: " + model_name)

llm=ChatOpenAI(model_name=model_name,
               openai_api_key=os.environ.get("TOGETHER_API_KEY"),
               openai_api_base="https://api.together.xyz/v1/")

import textwrap
from langchain_core.prompts import ChatPromptTemplate

def wrap_text(text, max_width=80):
    """
    Wraps the text to the specified max_width, preserving line breaks and formatting.
    """
    text = text.lstrip()
    lines = text.splitlines()  # Split the text into lines
    wrapped_lines = []
    for line in lines:
        if line.strip():  # Skip empty lines
            wrapped_line = textwrap.fill(line, max_width, initial_indent='', subsequent_indent='')
            wrapped_lines.extend(wrapped_line.splitlines())  # Preserve line breaks
        else:
            wrapped_lines.append('')  # Keep empty lines
    return '\n'.join(wrapped_lines)

def print_prompt_and_response(prompt, response):
    print("Prompt: ")
    print(wrap_text(prompt))
    print("")
    print("Response: ")
    print(response)

def print_messages_and_response(messages, response):
    prompt = ChatPromptTemplate(messages=messages)
    print_prompt_and_response(prompt.format(), response)

def get_completion(prompt, temperature=0.0):
    response = llm.invoke(prompt, temperature=temperature)
    wrapped_response = wrap_text(response.content)
    return wrapped_response

def get_completion_messages(messages, temperature=0.0):
    response=llm.invoke(messages, temperature=temperature)
    wrapped_response = wrap_text(response.content)
    return wrapped_response


### Description of Simple LLM Chain

Here we build a simple LLM Chain that makes two calls to the LLM. The
second step depends on the answer from the first step. This shows the
idea of a chain.

### Step 1: What is the city {person} is from?

In [ ]:
city_template="""
Where is {person} from? Give just the city name
"""

city_prompt = city_template.format(person="Sean Connery")
city_response = get_completion(city_prompt)
print(city_response)

### Step 2: What country is {city} in?

In [ ]:
country_template="""
What country is the city {city} in? Respond in {language} with just the name.
"""

country_prompt = country_template.format(city=city_response,
                                         language="French")
country_response = get_completion(country_prompt)
print(country_response)

### Step 3: Final Report

Sometimes we will want to print out a summary of the responses we get in
a chain. This may or may not include the prompts we give as well. We can
just use a template for this as well, even though this is not a prompt,
it is a convenient way to set up a printed version of the information.

In [ ]:
report_template="""
## Final Report

### Person
{person}

### City
{city}

### Language
{language}

### Country
{country}
"""

# We pass the format function the values we used or got from the LLM
report = report_template.format(person = "Sean Connery",
                                city = city_response,
                                language = "French",
                                country= country_response)
print(report)